<div>
<img src="figures/svtLogo.png"/>
</div>  

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Lab 8 - Elimination of variables, Penalty and SQP methods</h2></center>

$$\newcommand{\bx}{\mathbf{x}}$$
The following problem is given:
\begin{align*}
  \min_{\bx \in \mathbb{R}^2} \;\; & f(\bx)  \\
   \text{s.t.} \;\;& x_1+x_2=8,
\end{align*}

where $f(\bx) = - (x_1^2+x_2^2+4x_1x_2)$.

<u>Task 1</u>: Find the minimum of the function using variable elimination.
Check the second-order sufficient conditions for the unconstrained one-variable problem.

Another possibility is to use the following penalty function:
\begin{align*}
  	Q(\bx;\mu)=f(\bx)+\frac{1}{2\mu} (x_1+x_2-8)^2\,,
\end{align*}
with $\mu>0$ being a penalty parameter.
     
<br>
<u>Task 2</u>: Write down the first-order necessary condition of optimality for minimizing $Q$.

<u>Task 3</u>: What happens as $\mu \rightarrow 0$?  Complete the implementation of the quadratic penalty method below:

Also, report the eigenvalues and the condition number of the Hessian for each $\mu$.

In [ ]:
import numpy as np
import scipy.optimize as sp

# to calculate the gradient and Hessian of the objective function
from autograd import grad
from autograd import hessian

# to solve additionally using SLSQP solver, later on
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from math import inf

### Objective, constraint, quadratic penalty function, gradient and hessian

In [ ]:
def objective(X):
    x1, x2 = X[0], X[1]
    f = -(x1 ** 2 + x2 ** 2 + 4 * x1 * x2)
    return f

In [ ]:
def constraint(X):
    x1, x2 = X[0], X[1]
    c = x1 + x2 - 8
    return c

In [ ]:
def penaltyFunction(X, mu):
    x1, x2 = X[0], X[1]
    # add your code here
    return f

In [ ]:
def gradient_function(x, mu): 
    return [el.item() for el in grad(penaltyFunction, 0)(x, mu)]

In [ ]:
def hessian_function(x, mu): 
    return hessian(penaltyFunction, 0)(x, mu)

### Quadratic penalty method

In [ ]:
def qpm(x0, mu): 
    
    # get eigenvalues of the Hessian
    w, v = np.linalg.eig(hessian_function(x0, mu))
    
    # get condition number of the Hessian
    n = np.linalg.cond(hessian_function(x0, mu))
    
    # unconstrained optimization using BFGS method
    res = sp.minimize(penaltyFunction, x0, args=(mu), method='BFGS', jac=gradient_function)
    
    return w, n, res.x

In [ ]:
mu = 1
x0 = np.array([1.,1.])

# acceptable constraint violation at optimum
eps_viol = 1e-15
constraint_violation = True

it = 0

print ("{:<10} {:<10} {:<20} {:^20} {:^30}".format('iter','mu','minimum','condition nr.', 'constraint violation'))
while constraint_violation:
    it = it + 1
    
    w, n, xmin = qpm(x0,mu)
    print ("{:<10d} {:<10.3e} [{:^8.4f}, {:^8.4f}] {:<4} {:<20.2e} {:^20.3e}".format(it,mu,xmin[0],xmin[1],' ',n,constraint(xmin)))
    
    if constraint(xmin) <= eps_viol:
        constraint_violation = False 
    
    # update for next iteration (e.g. half of previous penalty value)
    # add your code here

### SLSQP method (scipy)

We will solve the problem using scipy's SLSQP solver (written by Dieter Kraft, DLR Oberpfaffenhofen)

In [ ]:
x0 = np.array([0.,0.])

bounds = Bounds([-inf,-inf], [inf,inf])

# The constraint is actually linear, so you can also try a different approach.
# See SLSQP documentation for more details on how to set up linear constraints.
nonlinear_constraints = NonlinearConstraint(constraint, 0, 0)

# use SLSQP
res = sp.minimize(# your code here
               constraints=[nonlinear_constraints], bounds=bounds, 
                  options={'disp': True, 'iprint': 4} )

print("minimum = {}".format(res.x))
print("constraint violation = {}".format(constraint(res.x)))
# The problem is a QP which is the reason why the SLSQP method is so fast. 

# Rosenbrock function contrained

The original Rosenbrock function does not have constraints, however, we introduce a constraint 
$$x_1^2 + x_2^2 - 1 \leq 0$$

In [ ]:
def rosenbrock(x):
    return ((x[0]-1)**2 + 100*(x[1]-x[0]**2)**2)

In [ ]:
def rosenbrock_inequality(x): 
    return x[0]**2 + x[1]**2 - 1

In [ ]:
def penalty_inequality(x, mu): 
    # add your code here
    return f

In [ ]:
def qpm_inequality(x0, mu): 
    
    # get eigenvalues of the Hessian
    #w, v = np.linalg.eig(hessian_function(x0, mu))
    
    # get condition number of the Hessian
    #n = np.linalg.cond(hessian_function(x0, mu))
    
    # unconstrained optimization using BFGS method
    res = sp.minimize(penalty_inequality, x0, args=(mu), method='BFGS')# jac=gradient_function)

    return res.x
    #return w, n, res.x

In [ ]:
mu = 1
x0 = np.array([0.0, 0.0])

# acceptable constraint violation at optimum
eps_f = 1e-8
sufficient_decrease = True

it = 0

print ("{:<10} {:<10} {:<20} {:^20} {:^30}".format('iter','mu','minimum','condition nr.', 'constraint value'))
while sufficient_decrease:
    it = it + 1
    f_prev = rosenbrock(x0)
    xmin = qpm_inequality(x0, mu)
    print ("{:<10d} {:<10.3e} [{:^8.4f}, {:^8.4f}] {:<4} {:<20.2e} {:^20.3e}".format(it,mu,xmin[0],xmin[1],' ',n, rosenbrock_inequality(xmin)))
    
    if abs(f_prev - rosenbrock(xmin)) <= eps_f:
        sufficient_decrease = False 
    
    # update penalty for next iteration (e.g. half of previous value)
    # add your code here


In [ ]:
x0 = np.array([0.,0.])

bounds = Bounds([-inf,-inf], [inf,inf])

nonlinear_constraints = NonlinearConstraint(rosenbrock_inequality, -inf, 0)

# use SLSQP
res = sp.minimize(# your code here
               constraints=[nonlinear_constraints], bounds=bounds, options={'disp': True, 'iprint': 4} )

print("minimum = {}".format(res.x))
print("constraint value = {}".format(rosenbrock_inequality(res.x)))